# openEO ArgoWorkflows Demonstration

## Setup

In [1]:
import os
import openeo
import json
import time
import xarray
import matplotlib.pyplot as pyplot
from IPython.display import display, Markdown

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [2]:
load_eoepca_state()

In [3]:
platform_domain = os.environ.get("INGRESS_HOST")

openeo_backend = f"openeo.{platform_domain}"
stac_api_url = f"https://eoapi.{platform_domain}/stac"

username = os.environ.get("KEYCLOAK_TEST_USER")
password = os.environ.get("KEYCLOAK_TEST_PASSWORD")
client_id = "openeo-argo"

collection_id = "sentinel-2-iceland"
temporal_extent = ["2023-11-09", "2023-11-10"]
spatial_extent = {"west": -24.9, "south": 64.2, "east": -24.5, "north": 64.5}

log_output_file = "openeo_argo_log.json"

In [4]:
print(f"Parameters:\n"
f"Platform domain: {platform_domain}\n"
f"OpenEO backend: {openeo_backend}\n"
f"STAC API URL: {stac_api_url}\n"
f"Client ID: {client_id}\n"
f"Collection ID: {collection_id}\n"
f"Temporal extent: {temporal_extent}\n"
f"Spatial extent: {spatial_extent}")

Parameters:
Platform domain: test.eoepca.org
OpenEO backend: openeo.test.eoepca.org
STAC API URL: https://eoapi.test.eoepca.org/stac
Client ID: openeo-argo
Collection ID: sentinel-2-iceland
Temporal extent: ['2023-11-09', '2023-11-10']
Spatial extent: {'west': -24.9, 'south': 64.2, 'east': -24.5, 'north': 64.5}


## Connect to the openEO ArgoWorkflows backend

In [5]:
connection = openeo.connect(url=openeo_backend)

# Authenticate using OIDC access token
access_token = get_access_token(username, password, client_id)
auth_token = f"oidc/eoepca/{access_token}"
connection.session.headers.update({"Authorization": f"Bearer {auth_token}"})

## Exploration

Available collections:

In [6]:
# collection_exists
available_collections = connection.list_collection_ids()
print(f"Available collections: {available_collections}")
assert collection_id in available_collections

Available collections: ['datacube-ready-collection', 'noaa-emergency-response', 'sentinel-2-datacube', 'sentinel-2-iceland', 'sentinel-2-l2a-datacube']


Collection details:

In [7]:
collection_info = connection.describe_collection(collection_id)
print(f"Collection: {collection_info.get('id')}")
print(f"Description: {collection_info.get('description', 'N/A')}")
print("\nAvailable bands:")
for band in collection_info.get("summaries", {}).get("eo:bands", []):
    print(f"  - {band.get('name')}: {band.get('common_name', 'N/A')}")

Collection: sentinel-2-iceland
Description: Sentinel-2 data from 2023 over Iceland.

Available bands:
  - coastal: coastal
  - blue: blue
  - green: green
  - red: red
  - rededge1: rededge
  - rededge2: rededge
  - rededge3: rededge
  - nir: nir
  - nir08: nir08
  - nir09: nir09
  - cirrus: cirrus
  - swir16: swir16
  - swir22: swir22


Available processes:

In [8]:
# list_processes
[p["id"] for p in connection.list_processes()[:10]]

['all',
 'pi',
 'not',
 'apply',
 'array_apply',
 'sar_backscatter',
 'arccos',
 'round',
 'ceil',
 'clip']

## Batch Job Processing

OpenEO ArgoWorkflows uses batch jobs for processing. We'll create a job, start it, wait for completion, then retrieve results.

### Load and process Sentinel-2 data

In [25]:
# Load collection using load_stac (pointing to our STAC catalogue)
stac_collection_url = f"{stac_api_url}/collections/{collection_id}"

cube = connection.load_stac(
    url=stac_collection_url,
    temporal_extent=temporal_extent,
    spatial_extent=spatial_extent,
    bands=["red", "nir"],
)

# Calculate NDVI
red = cube.band("red")
nir = cube.band("nir")
ndvi = (nir - red) / (nir + red)

# Save result as GTiff
result = ndvi.save_result(format="netCDF")

### Create and start the batch job

In [11]:
job = result.create_job(title="NDVI Iceland")
print(f"Created job: {job.job_id}")

Created job: 3fc1278b-f22f-4071-96dd-db774c163655


In [12]:
job.start_job()
print(f"Job started: {job.job_id}")

Job started: 3fc1278b-f22f-4071-96dd-db774c163655


### Monitor job progress

In [13]:
def wait_for_job(job, poll_interval=10, timeout=600):
    """Wait for a job to complete, polling at the specified interval."""
    start_time = time.time()
    
    while True:
        status = job.status()
        print(f"Job {job.job_id}: {status}")
        
        if status == "finished":
            print("Job completed successfully!")
            return True
        elif status in ["error", "canceled"]:
            print(f"Job failed with status: {status}")
            # Try to get error details
            job_info = job.describe_job()
            if "message" in job_info:
                print(f"Error message: {job_info['message']}")
            return False
        
        elapsed = time.time() - start_time
        if elapsed > timeout:
            print(f"Timeout after {timeout} seconds")
            return False
        
        time.sleep(poll_interval)

In [14]:
success = wait_for_job(job, poll_interval=15, timeout=300)

Job 3fc1278b-f22f-4071-96dd-db774c163655: running
Job 3fc1278b-f22f-4071-96dd-db774c163655: running
Job 3fc1278b-f22f-4071-96dd-db774c163655: finished
Job completed successfully!


### Download results

In [16]:
if success:
    results = job.get_results()
    print("Available result files:")
    for asset in results.get_assets():
        print(f"  - {asset.name}: {asset.href}")

Failed to parse API error response: [500] 'Internal Server Error' (headers: {'Content-Type': 'text/plain; charset=utf-8', 'Content-Length': '21', 'Connection': 'keep-alive', 'date': 'Thu, 18 Dec 2025 14:38:11 GMT', 'Server': 'APISIX', 'X-APISIX-Upstream-Status': '500', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, PATCH, OPTIONS', 'Access-Control-Max-Age': '5', 'Access-Control-Allow-Headers': 'DNT,X-CustomHeader,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Authorization'})


Available result files:


OpenEoApiPlainError: [500] Internal Server Error

In [ ]:
if success:
    # Download all results to a directory
    results.download_files("./openeo-argo-results/")
    print("Results downloaded to ./openeo-argo-results/")

### Inspect results

In [ ]:
import glob
import rasterio
from rasterio.plot import show

# Find downloaded GeoTIFF files
tiff_files = glob.glob("./openeo-argo-results/*.tif")
print(f"Found {len(tiff_files)} GeoTIFF files")

if tiff_files:
    # Display the first result
    with rasterio.open(tiff_files[0]) as src:
        fig, ax = pyplot.subplots(dpi=75, figsize=(8, 8))
        show(src, ax=ax, cmap="RdYlGn", vmin=-1, vmax=1)
        ax.set_title("NDVI - Iceland (Sentinel-2)")

### List all jobs

In [26]:
jobs = connection.list_jobs()
print(f"Total jobs: {len(jobs)}")
for j in jobs[:5]:
    print(f"  - {j.get('id')}: {j.get('status')} - {j.get('title', 'Untitled')}")

Failed to parse API error response: [500] 'Internal Server Error' (headers: {'Content-Type': 'text/plain; charset=utf-8', 'Content-Length': '21', 'Connection': 'keep-alive', 'date': 'Thu, 18 Dec 2025 14:47:38 GMT', 'Server': 'APISIX', 'X-APISIX-Upstream-Status': '500', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, PATCH, OPTIONS', 'Access-Control-Max-Age': '5', 'Access-Control-Allow-Headers': 'DNT,X-CustomHeader,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Authorization'})


OpenEoApiPlainError: [500] Internal Server Error

In [ ]:
if test_results:
    for test, result in test_results.items():
        print(f"{test}: {result['status']} - {result['message']}")
    json.dump(test_results, open(log_output_file, "w"), indent=2)